<a href="https://colab.research.google.com/github/Karagiannis-Giorg/A-new-perspective-to-Smart-Fridges/blob/main/Simulation/Simulation_F_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install roboflow
!pip install streamlit ngrok opencv-python-headless
!pip install pyngrok
!ngrok config add-authtoken 2rXJVXzr26fTTIW0Qj1torNd6Il_iiVaeFHCJtw5zki9ubVc

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

from detectron2.utils.logger import setup_logger
setup_logger()

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/My Drive/Georgios_Karagiannis/Code'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-2fkh6nbw
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-2fkh6nbw
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.2 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("level1_images")
version = project.version(2)
dataset = version.download("coco")

rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("smart-fridge-p5rw0")
version = project.version(9)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to level1_images-2 in coco:: 100%|██████████| 915/915 [00:00<00:00, 6517.11it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Smart-Fridge-9 in coco:: 100%|██████████| 1175/1175 [00:00<00:00, 6043.51it/s]


In [ ]:
%%writefile smart_fridge_app.py
import streamlit as st
import numpy as np
import glob
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
import json

# Καταχώριση των δεδομένων COCO και προσθήκη των κατηγοριών
def register_coco_instances_with_classes(dataset_name, json_file, image_dir):
    # Εγγραφή των δεδομένων COCO
    register_coco_instances(dataset_name, {}, json_file, image_dir)

    # Ανάγνωση του αρχείου json για να πάρουμε τις κατηγορίες
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    # Λήψη των κατηγοριών από το αρχείο COCO
    thing_classes = [category['name'] for category in coco_data['categories']]

    # Ενημέρωση του metadata για να περιλαμβάνει τις κατηγορίες
    metadata = MetadataCatalog.get(dataset_name)
    metadata.thing_classes = thing_classes

# Εγγραφή των δεδομένων για train, val, test και ενημέρωση των κατηγοριών
register_coco_instances_with_classes("my_dataset_train_level1", "/content/level1_images-2/train/_annotations.coco.json", "/content/level1_images-2/train")
register_coco_instances_with_classes("my_dataset_val_level1", "/content/level1_images-2/valid/_annotations.coco.json", "/content/level1_images-2/valid")
register_coco_instances_with_classes("my_dataset_test_level1", "/content/level1_images-2/test/_annotations.coco.json", "/content/level1_images-2/test")

register_coco_instances_with_classes("my_dataset_train_level2", "/content/Smart-Fridge-9/train/_annotations.coco.json", "/content/Smart-Fridge-9/train")
register_coco_instances_with_classes("my_dataset_val_level2", "/content/Smart-Fridge-9/valid/_annotations.coco.json", "/content/Smart-Fridge-9/valid")
register_coco_instances_with_classes("my_dataset_test_level2", "/content/Smart-Fridge-9/test/_annotations.coco.json", "/content/Smart-Fridge-9/test")


def setup_faster_rcnn_level1():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config1.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level1/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor1 = DefaultPredictor(cfg)
    metadata1 = MetadataCatalog.get("my_dataset_train_level1")
    return predictor1, metadata1

def setup_faster_rcnn_level2():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor2 = DefaultPredictor(cfg)
    metadata2 = MetadataCatalog.get("my_dataset_train_level2")
    return predictor2, metadata2

# Ανάλυση Εικόνας
def analyze_image(image, temperature, humidity, predictor1, predictor2, metadata1, metadata2):
    im = cv2.imread(image)
    outputs1 = predictor1(im)
    pred_classes_indices_level1 = outputs1["instances"].pred_classes.cpu().numpy()
    pred_classes_names_level1 = [metadata1.thing_classes[i] for i in pred_classes_indices_level1]

    if "Tomato" in pred_classes_names_level1:
        outputs2 = predictor2(im)
        pred_classes_indices_level2 = outputs2["instances"].pred_classes.cpu().numpy()
        pred_classes_names_level2 = [metadata2.thing_classes[i] for i in pred_classes_indices_level2]
        ripeness_message = []

        for ripeness in pred_classes_names_level2:
            if ripeness == "over-ripe":
                ripeness_message.append("Over-ripe tomatoes detected. Discard them immediatley before they start to effect the other fruits.")
            elif ripeness == "ripe":
                if 7 <= temperature <= 10 and humidity > 70:
                    ripeness_message.append("Ripe tomatoes detected. They are edible for up to 4 days. If you want your tomatoes to last longer, you should store them somewhere cooler.")
                elif temperature < 7:
                    ripeness_message.append("You should put your ripe tomatoes somewhere warmer to avoid chilling injuries.")
                else:
                    ripeness_message.append("YYour tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "half-ripe":
                if 8 <= temperature <= 10 and humidity > 60:
                    ripeness_message.append("Half-ripe tomatoes detected. They are not edible but can be stored for up to 7 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "unripe":
                if 8 <= temperature <= 12 and humidity > 50:
                    ripeness_message.append("Unripe tomatoes detected. They are not edible but can be stored for up to 10 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
        return ripeness_message
    else:
        remaining_items = [item for item in pred_classes_names_level1 if item != "Tomato"]
        return f"No tomatoes detected. Items in the fridge: {', '.join(remaining_items)}"

# Streamlit UI
def main():
    # Κεντρικός τίτλος
    st.markdown(
        "<h1 style='text-align: center;'>Welcome back</h1>",
        unsafe_allow_html=True,
    )
    # Εισαγωγή δεδομένων χρήστη
    temperature = st.number_input(
        "Temperature (°C)", value=8.0, min_value=-10.0, max_value=15.0, step=1.0
    )
    humidity = st.number_input(
        "Humidity (%)", value=75.0, min_value=0.0, max_value=100.0, step=1.0
    )

    st.markdown(
        "<h3 style='text-align: center;'>What would you like to do?</h3>",
        unsafe_allow_html=True,
    )

    # Αρχικοποίηση Session State για την εικόνα
    if "selected_image" not in st.session_state:
        simulation_image_path = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation"
        image_files = glob.glob(os.path.join(simulation_image_path, "*.[jp][pn]g"))
        if not image_files:
            st.error("No images found in the test directory.")
            st.stop()
        st.session_state.selected_image = random.choice(image_files)

    selected_image = st.session_state.selected_image

    if st.button("See what's inside"):
        # Εκτέλεση Faster R-CNN για ανάλυση περιεχομένων ψυγείου
        predictor1, metadata1 = setup_faster_rcnn_level1()
        im = cv2.imread(selected_image)

        # Εκτέλεση πρόβλεψης
        outputs1 = predictor1(im)

        # Οπτικοποίηση αποτελεσμάτων με Visualizer
        v = Visualizer(
            im[:, :, ::-1],  # Convert BGR to RGB
            metadata=metadata1,
            scale=0.8
        )
        out = v.draw_instance_predictions(outputs1["instances"].to("cpu"))

        # Μετατροπή της εικόνας από OpenCV (numpy array) σε μορφή PNG
        processed_img_path = "fridge_analysis.png"
        cv2.imwrite(processed_img_path, out.get_image()[:, :, ::-1])  # Convert RGB to BGR for OpenCV

        st.success("Analysis Complete!")
        # Εμφάνιση της εικόνας με ανιχνεύσεις
        st.image(processed_img_path, caption="Detected Items in the Fridge", use_container_width=True)

        # Υπολογισμός αριθμού αντικειμένων
        pred_classes_indices_level1 = outputs1["instances"].pred_classes.cpu().numpy()
        pred_classes_names_level1 = [metadata1.thing_classes[i] for i in pred_classes_indices_level1]
        item_counts = {item: pred_classes_names_level1.count(item) for item in set(pred_classes_names_level1)}

        st.write("Contents of the fridge:")
        for item, count in item_counts.items():
            st.write(f"- {count} {item}(s)")

    if st.button("See the maturity"):
        # Εκτέλεση Faster R-CNN για ανάλυση περιεχομένων ψυγείου
        predictor1, metadata1 = setup_faster_rcnn_level1()
        predictor2, metadata2 = setup_faster_rcnn_level2()
        im = cv2.imread(selected_image)

        # Εκτέλεση πρόβλεψης
        outputs = predictor2(im)

        # Οπτικοποίηση αποτελεσμάτων με Visualizer
        v = Visualizer(
            im[:, :, ::-1],  # Convert BGR to RGB
            metadata=metadata2,
            scale=0.8
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        # Μετατροπή της εικόνας από OpenCV (numpy array) σε μορφή PNG
        processed_img_path = "fridge_analysis.png"
        cv2.imwrite(processed_img_path, out.get_image()[:, :, ::-1])  # Convert RGB to BGR for OpenCV

        st.success("Analysis Complete!")
        # Εμφάνιση της εικόνας με ανιχνεύσεις
        st.image(processed_img_path, caption="Ripeness of tomatoes detected", use_container_width=True)

        # Ανάλυση ωριμότητας
        ripeness_analysis = analyze_image(selected_image, temperature, humidity, predictor1, predictor2, metadata1, metadata2)

        # Εμφάνιση ανάλυσης ωριμότητας
        if isinstance(ripeness_analysis, list):
            st.write("Ripeness Results:")
            for msg in ripeness_analysis:
                st.write("- " + msg)
        else:
            st.write(ripeness_analysis)

if __name__ == "__main__":
    main()

Overwriting smart_fridge_app.py


In [ ]:
from pyngrok import ngrok
import os

# Εκκίνηση Streamlit server
os.system("streamlit run smart_fridge_app.py &")

# Σύνδεση με ngrok
public_url = ngrok.connect(8501)  # Η θύρα πρέπει να είναι αριθμός χωρίς `port=`
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://e0db-34-168-49-131.ngrok-free.app" -> "http://localhost:8501"


# Αξιολόγηση του Pipeline

In [ ]:
# Δεδομένα σε coco format για την αξιολόγηση
from roboflow import Roboflow
rf = Roboflow(api_key="F357b98Kl2koA8RqsGzW")
project = rf.workspace("simulation-sv1vx").project("simulation-repenning")
version = project.version(2)
dataset = version.download("coco")

rf = Roboflow(api_key="F357b98Kl2koA8RqsGzW")
project = rf.workspace("simulation-sv1vx").project("simulation-dsrze")
version = project.version(1)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Simulation-repenning-2 in coco:: 100%|██████████| 27/27 [00:00<00:00, 5282.75it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Simulation-1 in coco:: 100%|██████████| 27/27 [00:00<00:00, 4833.38it/s]


In [ ]:
import numpy as np
import glob
import random
import os
import shutil

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
import json


# Καταχώριση των δεδομένων COCO και προσθήκη των κατηγοριών
def register_coco_instances_with_classes(dataset_name, json_file, image_dir):
    # Εγγραφή των δεδομένων COCO
    register_coco_instances(dataset_name, {}, json_file, image_dir)

    # Ανάγνωση του αρχείου json για να πάρουμε τις κατηγορίες
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    # Λήψη των κατηγοριών από το αρχείο COCO
    thing_classes = [category['name'] for category in coco_data['categories']]

    # Ενημέρωση του metadata για να περιλαμβάνει τις κατηγορίες
    metadata = MetadataCatalog.get(dataset_name)
    metadata.thing_classes = thing_classes

# Εγγραφή των δεδομένων για train, val, test και ενημέρωση των κατηγοριών
register_coco_instances_with_classes("my_dataset_train_level1", "/content/level1_images-2/train/_annotations.coco.json", "/content/level1_images-2/train")
register_coco_instances_with_classes("my_dataset_test_level1", "/content/Simulation-1/train/_annotations.coco.json", "/content/Simulation-1/train")

register_coco_instances_with_classes("my_dataset_train_level2", "/content/Smart-Fridge-9/train/_annotations.coco.json", "/content/Smart-Fridge-9/train")
register_coco_instances_with_classes("my_dataset_test_level2", "/content/Simulation-repenning-2/valid/_annotations.coco.json", "/content/Simulation-repenning-2/valid")

# Ρύθμιση Faster R-CNN
def setup_faster_rcnn_level1():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config1.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level1/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor1 = DefaultPredictor(cfg)
    metadata1 = MetadataCatalog.get("my_dataset_train_level1")
    return predictor1, metadata1,cfg

def setup_faster_rcnn_level2():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor2 = DefaultPredictor(cfg)
    metadata2 = MetadataCatalog.get("my_dataset_train_level2")
    return predictor2, metadata2,cfg

predictor, metadata, cfg1 = setup_faster_rcnn_level1()
predictor, metadata, cfg2 = setup_faster_rcnn_level2()
os.makedirs(cfg1.OUTPUT_DIR, exist_ok=True)

# Φόρτωση Faster R-CNN και αξιολόγηση
def evaluate_faster_rcnn(cfg, dataset_name):
    evaluator = COCOEvaluator(dataset_name, output_dir="./output")
    val_loader = build_detection_test_loader(cfg, dataset_name)
    results = inference_on_dataset(DefaultPredictor(cfg).model, val_loader, evaluator)
    return results

# Υπολογισμός συνολικής ακρίβειας
# (προσαρμόζεται ανάλογα με τη χρήση των μοντέλων)
def compute_overall_accuracy(faster_rcnn_metrics_1,faster_rcnn_metrics_2):
    faster_rcnn_map1 = faster_rcnn_metrics_1["bbox"]['AP']  # mAP του Faster R-CNN
    faster_rcnn_map2 = faster_rcnn_metrics_2["bbox"]['AP']  # mAP του Faster R-CNN
    print(f"Faster R-CNN mAP level1: {faster_rcnn_map1}")
    print(f"Faster R-CNN mAP level2: {faster_rcnn_map2}")
    overall_accuracy = (faster_rcnn_map1 + faster_rcnn_map2) / 2  # Μέσος όρος
    return overall_accuracy

fast_rcnn_levle1=evaluate_faster_rcnn(cfg1, "my_dataset_test_level1")
fast_rcnn_levle2=evaluate_faster_rcnn(cfg2, "my_dataset_test_level2")

overall_accuracy = compute_overall_accuracy(fast_rcnn_levle1, fast_rcnn_levle2)
print(f"Overall Accuracy: {overall_accuracy}")

[02/13 11:35:14 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level1/model_final.pth ...


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


[02/13 11:35:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth ...
WARNING [02/13 11:35:33 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/13 11:35:33 d2.data.datasets.coco]: Loaded 23 images in COCO format from /content/Simulation-1/train/_annotations.coco.json
[02/13 11:35:33 d2.data.build]: Distribution of instances among all 9 categories:
|  category  | #instances   |  category   | #instances   |  category  | #instances   |
|:----------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
|  objects   | 0            |    Apple    | 15           |   Banana   | 10           |
|    Lime    | 7            |   Orange    | 5            |   Peach    | 6            |
|    Pear    | 3            | Pomegranate | 9            |   Tomato   | 49           |
|            |              |             |     

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)


[02/13 11:35:39 d2.evaluation.evaluator]: Inference done 11/23. Dataloading: 0.0013 s/iter. Inference: 0.1005 s/iter. Eval: 0.0003 s/iter. Total: 0.1021 s/iter. ETA=0:00:01
[02/13 11:35:40 d2.evaluation.evaluator]: Total inference time: 0:00:01.876343 (0.104241 s / iter per device, on 1 devices)
[02/13 11:35:40 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.099944 s / iter per device, on 1 devices)
[02/13 11:35:40 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[02/13 11:35:40 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[02/13 11:35:40 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[02/13 11:35:40 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[02/13 11:35:40 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[02/13 11:35:40 d2.evaluation.

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


[02/13 11:35:43 d2.evaluation.evaluator]: Start inference on 23 batches
[02/13 11:35:44 d2.evaluation.evaluator]: Inference done 11/23. Dataloading: 0.0084 s/iter. Inference: 0.1078 s/iter. Eval: 0.0005 s/iter. Total: 0.1168 s/iter. ETA=0:00:01
[02/13 11:35:46 d2.evaluation.evaluator]: Total inference time: 0:00:02.119468 (0.117748 s / iter per device, on 1 devices)
[02/13 11:35:46 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.106144 s / iter per device, on 1 devices)
[02/13 11:35:46 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[02/13 11:35:46 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[02/13 11:35:46 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[02/13 11:35:46 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[02/13 11:35:46 d2.evaluation.fast_eval_api]: COCOeval